In [5]:
# https://stackoverflow.com/questions/5099193/using-beautifulsoup-to-find-all-elements-starting-with-a-given-letter 

# undetected chromedriver
# #!/usr/bin/python3

# import pandas as pd
# import undetected_chromedriver as uc
# from bs4 import BeautifulSoup
# import selenium
# from selenium.webdriver.chrome.options import Options

# import time
# import sys
# import os
# import numpy as np

# options = Options()
# # options.add_argument('--incognito')
# driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=options)

# https://www.zenrows.com/blog/undetected-chromedriver#how-to-use-undetected-chromedriver 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import os
import selenium
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import random
import undetected_chromedriver as uc

In [ ]:
# Nusirašyti tamstai reikia tris funkcijas: getColNamesList, alterTableAddColumn ir writeMany

In [2]:
# Susirenkame šaldytuvų nuorodas.

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)

nuorodos=[]

for i in range(1, 34):
    url = f'https://www.varle.lt/saldytuvai/?p={i}' 
    driver.get(url)
    time.sleep(2)
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    varle = bs.find_all('div', {'class':'GRID_ITEM'})

    for saldytuvas in varle:
        try:
            nuorodos.append(saldytuvas.find('div', {'class':'product-title'}).find('a')['href'])
        except:
            pass
        
driver.close()

In [40]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 1'''.format(table=table)
    s = c.execute(sql)
    r = s.description
    return r

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(0.25)
            db.commit()
            # time.sleep(3)
    c.close()
    db.close()
    return True

def writeMany(data, table, dbname):
    connection = sqlite3.connect(dbname)
    cursor = connection.cursor()

    # Insert dictionary data into the table
    # columns = ', '.join(data.keys())  # Get column names as a string
    placeholders = ', '.join(['?'] * len(data))  # Generate placeholders for values
    values = tuple(data.values())  # Get values as a tuple

    # Construct the SQL INSERT statement
    sql = f"INSERT INTO {table} VALUES ({placeholders})"
    cursor.execute(sql, values)

    # Commit the changes and close the connection
    connection.commit()
    cursor.close()
    connection.close()
    return True


SDB = sqlite3.connect('varlesaldytuvai.db') # jei neegsiztuoja - bus sukurfta nauja sqlite DB
Cs = SDB.cursor()

sql = '''create table if not exists saldytuvai
(
gamintojas text,
nuoroda text
)
 '''
Cs.execute(sql)
Cs.close()
SDB.close()



opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)

for nuor in range(180, len(nuorodos)):
    try:
        url = f'https://www.varle.lt{nuorodos[nuor]}'
        driver.get(url)
        time.sleep(2)
        source = driver.page_source
        bs = BeautifulSoup(source, 'html.parser')
        specifikacija = bs.find('div', {'class' : 'accordion'}).find('div', {'class': 'PRODUCT_SPEC'})
        duom=specifikacija.find_all('li', {'class': 'spec-line'})

        stulpeliai, *_ =zip(*getColNamesList('saldytuvai', 'varlesaldytuvai.db'))
        duomenys = {key: None for key in stulpeliai}
        duomenys['nuoroda']= nuorodos[nuor]
    
        kainaa = bs.find('div', {'class' : 'container'})
        kainaa= kainaa.find('span', {'class': 'price-value'})['content'].strip()
        duomenys['kaina'] = kainaa
    
        if len(duom) > 0:
            for i in duom:
                try:
                    stulpeliai, *_ =zip(*getColNamesList('saldytuvai', 'varlesaldytuvai.db'))
                    # print(i.find('span', {'class':'left-column'}).text.lower(), ' : ', i.find('span', {'class':'right-column'}).text.lower())
                    kas = i.find('span', {'class':'left-column'}).text.lower().strip()
                    kaip = i.find('span', {'class':'right-column'}).text.lower().strip()
                    duomenys[kas]= kaip
                except:
                    pass
        else:
            specifikacija1 = bs.find('div', {'class' : 'accordion'}).find('div', {'class': 'PRODUCT_SPEC'})
            duom1=specifikacija1.find_all('tr')

            for j in duom1:
                if len(j) == 2:
                    kas = j.find('td', {'class': 'key'}).text.lower().strip()
                    kaip = j.find('td', {'class': 'value'}).text.lower().strip()
                    duomenys[kas]= kaip
    
        try:
            specifikacija2 = bs.find('div', {'class' : 'accordion'}).find('div', {'class': 'PRODUCT_SPEC_FILTERS'})
            specifikacija2= specifikacija2.find_all('li', {'class': 'spec-line'})
            for i in specifikacija2:
                kas = i.find('span', {'class': 'left-column'}).text.lower().strip()
                kaip = i.find('span', {'class': 'right-column'}).text.lower().strip()
                duomenys[kas]=kaip
        except:
            pass
        
    except:
        stulpeliai, *_ =zip(*getColNamesList('saldytuvai', 'varlesaldytuvai.db'))
        alterTableAddColumn(duomenys.keys(), stulpeliai, 'saldytuvai', 'varlesaldytuvai.db')
        pass
    else:
        stulpeliai, *_ =zip(*getColNamesList('saldytuvai', 'varlesaldytuvai.db'))
        alterTableAddColumn(duomenys.keys(), stulpeliai, 'saldytuvai', 'varlesaldytuvai.db')
        writeMany(duomenys, 'saldytuvai', 'varlesaldytuvai.db')
    

driver.close()
    

In [3]:
# Susirenkame varle.lt televizorių nuorodas.

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)

nuorodos=[]

for i in range(1, 31):
    url = f'https://www.varle.lt/televizoriai/?p={i}' 
    driver.get(url)
    time.sleep(2)
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    varle = bs.find_all('div', {'class':'GRID_ITEM'})

    for dronai in varle:
        try:
            nuorodos.append(dronai.find('div', {'class':'product-title'}).find('a')['href'])
        except:
            pass
        
driver.close()

In [4]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 1'''.format(table=table)
    s = c.execute(sql)
    r = s.description
    return r

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(0.25)
            db.commit()
            # time.sleep(3)
    c.close()
    db.close()
    return True

def writeMany(data, table, dbname):
    connection = sqlite3.connect(dbname)
    cursor = connection.cursor()

    # Insert dictionary data into the table
    # columns = ', '.join(data.keys())  # Get column names as a string
    placeholders = ', '.join(['?'] * len(data))  # Generate placeholders for values
    values = tuple(data.values())  # Get values as a tuple

    # Construct the SQL INSERT statement
    sql = f"INSERT INTO {table} VALUES ({placeholders})"
    cursor.execute(sql, values)

    # Commit the changes and close the connection
    connection.commit()
    cursor.close()
    connection.close()
    return True


SDB = sqlite3.connect('varletelevizoriai.db') # jei neegsiztuoja - bus sukurfta nauja sqlite DB
Cs = SDB.cursor()

sql = '''create table if not exists televizoriai
(
gamintojas text,
nuoroda text
)
 '''
Cs.execute(sql)
Cs.close()
SDB.close()



opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)

for nuor in range(0, len(nuorodos)):
    try:
        url = f'https://www.varle.lt{nuorodos[nuor]}'
        driver.get(url)
        time.sleep(2)
        source = driver.page_source
        bs = BeautifulSoup(source, 'html.parser')
        specifikacija = bs.find('div', {'class' : 'accordion'}).find('div', {'class': 'PRODUCT_SPEC'})
        duom=specifikacija.find_all('li', {'class': 'spec-line'})

        stulpeliai, *_ =zip(*getColNamesList('televizoriai', 'varletelevizoriai.db'))
        duomenys = {key: None for key in stulpeliai}
        duomenys['nuoroda']= nuorodos[nuor]
    
        kainaa = bs.find('div', {'class' : 'container'})
        kainaa= kainaa.find('span', {'class': 'price-value'})['content'].strip()
        duomenys['kaina'] = kainaa
    
        if len(duom) > 0:
            for i in duom:
                try:
                    stulpeliai, *_ =zip(*getColNamesList('televizoriai', 'varletelevizoriai.db'))
                    # print(i.find('span', {'class':'left-column'}).text.lower(), ' : ', i.find('span', {'class':'right-column'}).text.lower())
                    kas = i.find('span', {'class':'left-column'}).text.lower().strip()
                    kaip = i.find('span', {'class':'right-column'}).text.lower().strip()
                    duomenys[kas]= kaip
                except:
                    pass
        else:
            specifikacija1 = bs.find('div', {'class' : 'accordion'}).find('div', {'class': 'PRODUCT_SPEC'})
            duom1=specifikacija1.find_all('tr')

            for j in duom1:
                if len(j) == 2:
                    kas = j.find('td', {'class': 'key'}).text.lower().strip()
                    kaip = j.find('td', {'class': 'value'}).text.lower().strip()
                    duomenys[kas]= kaip
    
        try:
            specifikacija2 = bs.find('div', {'class' : 'accordion'}).find('div', {'class': 'PRODUCT_SPEC_FILTERS'})
            specifikacija2= specifikacija2.find_all('li', {'class': 'spec-line'})
            for i in specifikacija2:
                kas = i.find('span', {'class': 'left-column'}).text.lower().strip()
                kaip = i.find('span', {'class': 'right-column'}).text.lower().strip()
                duomenys[kas]=kaip
        except:
            pass
        
    except:
        stulpeliai, *_ =zip(*getColNamesList('televizoriai', 'varletelevizoriai.db'))
        alterTableAddColumn(duomenys.keys(), stulpeliai, 'televizoriai', 'varletelevizoriai.db')
        pass
    else:
        stulpeliai, *_ =zip(*getColNamesList('televizoriai', 'varletelevizoriai.db'))
        alterTableAddColumn(duomenys.keys(), stulpeliai, 'televizoriai', 'varletelevizoriai.db')
        writeMany(duomenys, 'televizoriai', 'varletelevizoriai.db')
    

driver.close()